# 🎓 Agentic AI: University Academic Policy Assistant

In [ ]:
!pip install openai langchain pymongo faiss-cpu

In [ ]:
academic_policy_text = """Students must complete a minimum of 120 credits to graduate, with at least 40 credits in major-specific courses. Each semester, students must register for a minimum of 12 credits and a maximum of 24.

Students must maintain a CGPA of 6.0 or above for good academic standing. A CGPA below 5.0 for two consecutive semesters may result in academic probation. Attendance must be at least 75% in each course to be eligible for final exams.

Students are allowed to retake failed courses or improve grades in up to 4 subjects across their academic tenure. Re-examinations are permitted only for genuine reasons and require approval from the academic dean.

Internships are mandatory for all students in their 6th semester and must be registered officially through the university portal. Any case of plagiarism in submitted assignments will lead to disciplinary action as per university policy.

Students with outstanding dues will not be allowed to register for the next semester or receive transcripts until dues are cleared."""

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

os.environ['OPENAI_API_KEY'] = 'your_openai_api_key_here'
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = splitter.create_documents([academic_policy_text])
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding)

In [ ]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')
db = client['university']
students = db['students']

def check_graduation_eligibility(name):
    s = students.find_one({"name": name})
    if not s: return 'Student not found.'
    return 'Eligible for graduation.' if s['credits_completed'] >= 120 else 'Not eligible yet.'

def get_cgpa_status(name):
    s = students.find_one({"name": name})
    if not s: return 'Student not found.'
    if s['cgpa'] >= 6.0:
        return f"CGPA is {s['cgpa']}. In good standing."
    elif s['cgpa'] < 5.0:
        return f"CGPA is {s['cgpa']}. May be on academic probation."
    else:
        return f"CGPA is {s['cgpa']}. Needs improvement."

def check_attendance(name, course):
    s = students.find_one({"name": name})
    if not s: return 'Student not found.'
    attendance = s.get('attendance', {}).get(course)
    if attendance is None: return f'No attendance record for {course}.'
    return f"Attendance for {course} is {attendance}%. {'Eligible' if attendance >= 75 else 'Not eligible'} for final exam."

def check_dues(name):
    s = students.find_one({"name": name})
    if not s: return 'Student not found.'
    return 'No dues pending.' if not s['dues_pending'] else 'Dues are pending.'

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent

llm = ChatOpenAI(model_name='gpt-4o')
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

tools = [
    Tool(name='PolicyQA', func=qa_chain.run, description='Answer academic policy questions'),
    Tool(name='CheckGraduationEligibility', func=lambda x: check_graduation_eligibility('Alice'), description='Check graduation credit status'),
    Tool(name='GetCGPAStatus', func=lambda x: get_cgpa_status('Alice'), description='Fetch CGPA and standing'),
    Tool(name='CheckAttendance', func=lambda x: check_attendance('Alice', 'ENG100'), description='Check attendance status for course'),
    Tool(name='CheckDues', func=lambda x: check_dues('Alice'), description='Check if student has pending dues')
]

agent = initialize_agent(tools=tools, agent='zero-shot-react-description', llm=llm, verbose=True)

In [ ]:
agent.run('How many credits do I need to graduate?')
agent.run('I am Alice, what is my CGPA and am I in good standing?')
agent.run('Can I take the final exam in ENG100?')
agent.run('Do I have any pending dues?')

In [ ]:
students.delete_many({})
students.insert_one({
    "student_id": "UG20231001",
    "name": "Alice",
    "cgpa": 6.8,
    "credits_completed": 110,
    "attendance": {
        "Math101": 82,
        "CS201": 76,
        "ENG100": 65
    },
    "dues_pending": False,
    "courses_retaken": ["CS101", "PHY102"]
})